# Отчёт по сущностям и потокам данных (KХД)

Цель: подготовить аналитический отчёт по сущностям, источникам и процессам сбора данных:
- Источник данных
- Ответственный за корректность
- Регламент и периодичность
- Инструмент сбора, степень автономности/автоматизации

Подход:
- Парсим таблицу `essence_data` и выделяем сущности (групповая сущность, наименование).
- Нормализуем ключевые поля (источники, регламенты, ответственные) для агрегирования.
- Строим сводки по сущностям и источникам, выявляем дубли/пробелы.
- Визуализируем поток данных от источников к сущностям (Mermaid diagram).

Артефакты:
- Сводные таблицы в ноутбуке
- Диаграмма потоков (Mermaid)
- Экспорт в файл `essence_report.md`

Примечание: отчёт структурирован по сущностям и потокам данных.

In [16]:
# Загрузка Excel: лист «Для Кирилла»
from pathlib import Path
import pandas as pd

# ищем файл
xls_candidates = [
    Path('data') / 'essence.xlsx',
    Path('..') / 'data' / 'essence.xlsx',
    Path('../..') / 'data' / 'essence.xlsx',
]
for p in xls_candidates:
    if p.is_file():
        XLS_PATH = p
        break
else:
    raise FileNotFoundError('Не найден essence.xlsx')

print('Использую файл:', XLS_PATH.resolve())

# загрузка нужного листа
essence_data = pd.read_excel(XLS_PATH, sheet_name='Для Кирилла')
print('Форма данных:', essence_data.shape)
print('Колонки:', list(essence_data.columns))
essence_data.head()

Использую файл: /Users/kirilltishchenko/lessonsPandas/data/essence.xlsx
Форма данных: (178, 13)
Колонки: ['№', 'Сущность', 'Показатель', '№ источника', 'Источник: система / таблица / поле', 'Инстурмент (результат)', 'Расположение источника', 'Ответственный', 'Описание / смысл', 'Частота обновления/источника', 'Инструмент сбора', 'Класс данных', 'Регламент сбора']


,№,Сущность,Показатель,№ источника,Источник: система / таблица / поле,Инстурмент (результат),Расположение источника,Ответственный,Описание / смысл,Частота обновления/источника,Инструмент сбора,Класс данных,Регламент сбора
0,1,МиМ,Часы МиМ,1.1,Отчет по ежедневным заявкам (факт),Дашборд по МиМ,\\nas\Data\Дирекция Механизации\Ежедневные заявки,Волынец,Основной показатель работы техники,потребность в еженедельном обновлении,скрипт python,обычные,NaN
1,1,МиМ,Часы МиМ,1.2,Отчет по месячный заявкам (план),Дашборд по МиМ,\\nas\Data\Дирекция Механизации\Ежедневные заявки,Волынец,Основной показатель работы техники,Ежемесячно,скрипт python,обычные,NaN
2,1,МиМ,Часы МиМ,1.3,Отчет по месячный заявкам (факт),Дашборд по МиМ,\\nas\Data\Дирекция Механизации\Ежедневные заявки,Волынец,Основной показатель работы техники,Ежемесячно,скрипт python,обычные,NaN
3,1,МиМ,Часы МиМ,1.4,Отчет SP (план + факт),Дашборд по МиМ,\\nas\Data\Графики SP\Резервные\Для аналитиков...,Цивилев,Основной показатель работы техники,Ежемесячно,скрипт python,обычные,NaN
4,1,МиМ,Наименование техники,1.1,Отчет по ежедневным заявкам (факт),Дашборд по МиМ,\\nas\Data\Дирекция Механизации\Ежедневные заявки,Волынец,Основной справочник единиц техники,потребность в еженедельном обновлении,скрипт python,обычные,NaN


In [13]:
# Сводки по сущностям и источникам (без переименования колонок)
import pandas as pd
import numpy as np

ess = essence_data.copy()

# Унификация пробелов
for col in ['Сущность', 'Показатель', 'Источник: система / таблица / поле', 'Ответственный',
            'Частота обновления/источника', 'Инструмент сбора', 'Класс данных', 'Регламент сбора']:
    if col in ess.columns:
        ess[col] = ess[col].astype(str).str.strip()

# Парсинг списка источников: разделители ; , /
if 'Источник: система / таблица / поле' in ess.columns:
    ess['__sources'] = ess['Источник: система / таблица / поле'] \
        .str.split(r'[;,/]+', regex=True) \
        .apply(lambda xs: [x.strip() for x in xs] if isinstance(xs, list) else [])
else:
    ess['__sources'] = [[] for _ in range(len(ess))]

# Ключ сущности (можно расширить номером при необходимости)
entity_key = ess['Сущность'].fillna('')

# Сводка по сущностям (все строки)
summary_entities = pd.DataFrame({
    'Сущность': entity_key,
    'Показателей, шт': ess.groupby(entity_key)['Показатель'].transform('nunique') if 'Показатель' in ess else 0,
    'Источники (уник.)': ess['__sources'].apply(set),
})
summary_entities = summary_entities.groupby('Сущность', as_index=False).agg({
    'Показателей, шт': 'max',
    'Источники (уник.)': lambda col: sorted(set().union(*col)) if len(col)>0 else [],
})
summary_entities['Число источников'] = summary_entities['Источники (уник.)'].apply(len)
summary_entities = summary_entities.sort_values(['Число источников','Показателей, шт','Сущность'],
                                                ascending=[False, False, True], ignore_index=True)
print(f"Всего сущностей: {len(summary_entities)}")
summary_entities

Всего сущностей: 3


,Сущность,"Показателей, шт",Источники (уник.),Число источников
0,Финансовые показатели,69,"[Spider, БФ-9, В процессе исправления ошибок в...",5
1,Персонал,4,"[Выгрузка из 1С, Выгрузка из СКУД, Расширенная...",5
2,МиМ,3,"[Отчет SP (план + факт), Отчет по ежедневным з...",4


In [32]:
# Группировка по источнику: считаем количество записей, уникальные сущности и показатели
copy_data = essence_data.copy()

# В исходном листе может не быть колонки 'Источник' — источники мы развернули в pairs_df
if 'Источник' in copy_data.columns:
    grp = (copy_data
           .groupby('Источник', dropna=False)
           .agg(Строк_в_исходнике=('Сущность', 'size'),
                Уник_сущностей=('Сущность', 'nunique'),
                Уник_показателей=('Показатель', 'nunique'))
           .sort_values('Строк_в_исходнике', ascending=False))
else:
    # Используем pairs_df, где уже есть колонка 'Источник'
    tmp = pairs_df.copy()
    grp = (tmp
           .groupby('Источник', dropna=False)
           .agg(Связей=('Сущность', 'size'),
                Уник_сущностей=('Сущность', 'nunique'),
                Уник_показателей=('Показатель', lambda s: s.nunique() if 'Показатель' in tmp.columns else 0))
           .sort_values('Связей', ascending=False))

grp.head(50)

,Связей,Уник_сущностей,Уник_показателей
Источник,,,
БФ-9,139,1,67
Spider,15,1,13
Выгрузка из 1С,4,1,3
Производственный_план-факт_2025_года.xlsx,4,1,2
Отчет SP (план + факт),3,1,3
Отчет по месячный заявкам (план),3,1,3
Отчет по месячный заявкам (факт),3,1,3
Отчет по ежедневным заявкам (факт),2,1,2
В процессе исправления ошибок в доставке данных из 1С в КХД,1,1,1


In [44]:
# Вывести уникальные значения столбца "Показатель" (отсортированный плоский список)
copy_data = essence_data.copy()

if 'Показатель' in copy_data.columns:
    uniq = (copy_data['Показатель']
            .dropna()
            .astype(str)
            .str.strip()
            .replace({'': None})
            .dropna()
            .unique())
    uniq_sorted = sorted(set(uniq))
    print(f"Всего уникальных показателей: {len(uniq_sorted)}")
    for i, v in enumerate(uniq_sorted, 1):
        print(f"{i:3}. {v}")
else:
    print("В исходных данных нет колонки 'Показатель'.")


Всего уникальных показателей: 75
  1. 0. Затраты Невыборка
  2. БДДС---Оплата ГрПП---Материалы и оборудование поставки ГрПП
  3. БДДС---Оплата ГрПП---Расходы на персонал
  4. БДДС---Оплата аванса---Оплата аванса
  5. БДДС---Оплаты по проекту УФК---Оплаты по проекту УФК
  6. БДДС---Оплаты по проекту---Аренда
  7. БДДС---Оплаты по проекту---ИТ, СРО, Лицензии, Консалтинг
  8. БДДС---Оплаты по проекту---Командировочные расходы
  9. БДДС---Оплаты по проекту---Материалы и оборудование поставки ГрПП
 10. БДДС---Оплаты по проекту---Материалы и оборудование прочих поставщиков
 11. БДДС---Оплаты по проекту---Машины и механизмы
 12. БДДС---Оплаты по проекту---Налоги, взносы, сборы, пени, штрафы
 13. БДДС---Оплаты по проекту---Оплаты по проекту
 14. БДДС---Оплаты по проекту---Оформление разрешительной документации
 15. БДДС---Оплаты по проекту---Погашение основного долга по займам
 16. БДДС---Оплаты по проекту---Прочие расходы
 17. БДДС---Оплаты по проекту---Расходы на персонал
 18. БДДС---Оплаты 